<a href="https://colab.research.google.com/github/zampierilucas/Download-to-Google-Drive/blob/master/Url_download_to_GDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install pySmartDL rarfile

In [ ]:
import requests
import sys
import time
import os
import rarfile
import shutil
import zipfile
from google.colab import drive
from io import StringIO
from pySmartDL import SmartDL


def extract_compressed(source, dest):
  print(f'Extracting {source.split("/")[-1]}')
  
  if source.endswith(".zip"):
    with zipfile.ZipFile(source, "r") as zip_ref:
      zip_ref.extractall(path)

  else:
    with rarfile.RarFile(source, "r") as rar_ref:
      rar_ref.extractall(path)

  os.remove(source)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)



downloads = []
path = "/content/drive/My Drive/Downloads/"

if not os.path.exists(path):
    os.makedirs(path)

while True:
    link = input("Enter URL Or Type Exit: ")
    if link.lower() == "exit":
        break
    downloads.append(link)


while downloads:
    print("Downloading\n")
    # for index, url in enumerate(downloads[:]):
    file_name = (url.split("/")[-1]).replace("%20"," ")

    obj = SmartDL(downloads, progress_bar=False)
    obj.start(blocking=False)

    while not obj.isFinished():
      if obj.get_status() == "downloading":
        sys.stdout.write(f"\rDownloading {file_name} {obj.get_progress_bar()} {obj.get_progress()*100:.2f}% {obj.get_dl_size(human=True)} {obj.get_speed(human=True)}   ")
        time.sleep(0.5)

      else:
        sys.stdout.write(f'\rCombining {file_name}')
        time.sleep(1)

      if obj.isSuccessful():
        sys.stdout.write(f"\rCompleted {file_name} File size: {obj.get_dl_size(human=True)} Time taken: {obj.get_dl_time(human=True)}   \n")

        if file_name.endswith(('.zip', '.rar')):
          extract_compressed(obj.get_dest(), path)
        else:
          file_name = obj.get_dest().split("/")[-1]
          print(path+file_name)
          if os.path.exists(path+file_name):
            print("removeing")
            os.remove(path+file_name)
          
          shutil.move(obj.get_dest(),path)
          
    else:
        print("There were some errors:")
        for e in obj.get_errors():
                print(str(e))

    # downloads.remove(url)
    
    print("\nAll downloads have been complete")

Downloading


RuntimeError: ignored